In [2]:
import pandas as pd
import numpy as np
import itertools
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from statsmodels.formula.api import ols

In [3]:
data = pd.read_csv("prostate.data",delimiter="\t")
data2=data.drop(["Unnamed: 0","train"],axis=1)

In [4]:
y_train = np.array(data[data.train == "T"]['lpsa'])
y_test = np.array(data[data.train == "F"]['lpsa'])
X_train = np.array(data[data.train == "T"].drop(['lpsa', 'train', 'Unnamed: 0'], axis=1))
X_test = np.array(data[data.train == "F"].drop(['lpsa', 'train', 'Unnamed: 0'], axis=1))

# Best subset selection

In [5]:
results = pd.DataFrame(columns=['num_features', 'features', 'R2_score'])
for k in range(1, X_train.shape[1] + 1):
    for subset in itertools.combinations(range(X_train.shape[1]), k):
        subset = list(subset)
        linreg_model = LinearRegression(normalize=True).fit(X_train[:, subset], y_train)
        linreg_prediction = linreg_model.predict(X_train[:, subset])
        linreg_score = np.mean(np.abs(r2_score(y_train,linreg_prediction)))
        results = results.append(pd.DataFrame([{'num_features': k,
                                                'features': subset,
                                                'R2_score': linreg_score}]))

results = results.sort_values('R2_score',ascending=False).reset_index()
results.drop( labels="index" , axis = 1 ,inplace=True)
print(results.groupby("num_features")["R2_score"].max())
best_subset_model = LinearRegression(normalize=True).fit(X_train[:, results['features'][0]], y_train)
best_subset_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[1:-1],
        np.round(np.concatenate((best_subset_model.intercept_, best_subset_model.coef_), axis=None), 3))
)

print('Best Subset Regression R2_score: {}'.format(np.round(results['R2_score'][0], 3)),'Best Subset Regression Model: {}'.format([data2.columns[i] for i in results["features"][0]]))
print('Best Subset Regression coefficients:', best_subset_coefs)

num_features
1    0.537516
2    0.614756
3    0.637441
4    0.659176
5    0.666920
6    0.682807
7    0.694258
8    0.694371
Name: R2_score, dtype: float64
Best Subset Regression R2_score: 0.694 Best Subset Regression Model: ['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']
Best Subset Regression coefficients: {'Intercept': 0.429, 'lcavol': 0.577, 'lweight': 0.614, 'age': -0.019, 'lbph': 0.145, 'svi': 0.737, 'lcp': -0.206, 'gleason': -0.03, 'pgg45': 0.009}


# Forward selection

In [22]:
def forward_selection(data,target):
    variate=data.columns.to_list()
    variate.remove(target)  
    selected=[]
    current_score,best_new_score=float('inf'),float('inf')  
    while variate:
        bic_with_variate=[]
        for feature in variate:  
            formula="{}~{}".format(target,"+".join(selected+[feature]))
            print(formula)
            bic=ols(formula=formula,data=data).fit().bic  
            bic_with_variate.append((bic,feature))  
        bic_with_variate.sort(reverse=True)  
        best_new_score,best_feature=bic_with_variate.pop()  
        
        if current_score >= best_new_score:  
            variate.remove(best_feature)  
            selected.append(best_feature)  
            current_score=best_new_score  
        else:
            print("forward selection is over!")
            break
            
    formula="{}~{}".format(target,"+".join(selected))  
    print("final formula is {}".format(formula))
    model=ols(formula=formula,data=data).fit()
    return(model)

fwd = forward_selection(data=data2,target='lpsa')
fwd.summary()

lpsa~lcavol
lpsa~lweight
lpsa~age
lpsa~lbph
lpsa~svi
lpsa~lcp
lpsa~gleason
lpsa~pgg45
lpsa~lcavol+lweight
lpsa~lcavol+age
lpsa~lcavol+lbph
lpsa~lcavol+svi
lpsa~lcavol+lcp
lpsa~lcavol+gleason
lpsa~lcavol+pgg45
lpsa~lcavol+lweight+age
lpsa~lcavol+lweight+lbph
lpsa~lcavol+lweight+svi
lpsa~lcavol+lweight+lcp
lpsa~lcavol+lweight+gleason
lpsa~lcavol+lweight+pgg45
lpsa~lcavol+lweight+svi+age
lpsa~lcavol+lweight+svi+lbph
lpsa~lcavol+lweight+svi+lcp
lpsa~lcavol+lweight+svi+gleason
lpsa~lcavol+lweight+svi+pgg45
forward selection is over!
final formula is lpsa~lcavol+lweight+svi


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   lpsa   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.624
Method:                 Least Squares   F-statistic:                     54.15
Date:                Tue, 11 May 2021   Prob (F-statistic):           2.44e-20
Time:                        13:30:53   Log-Likelihood:                -102.05
No. Observations:                  97   AIC:                             212.1
Df Residuals:                      93   BIC:                             222.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.7772      0.623     -1.247      0.215      -2.014       0.460
lcavol         0.5259      0.075      7.024      0.000       0.377       0.675
lweight        0.6618      0.176      3.768      0.000       0.313       1.011
svi            0.6657      0.207      3.214      0.002       0.254       1.077
==============================================================================
Omnibus:                        0.099   Durbin-Watson:                   1.501
Prob(Omnibus):                  0.952   Jarque-Bera (JB):                0.215
Skew:                          -0.069   Prob(JB):                        0.898
Kurtosis:                       2.815   Cond. No.                         36.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Backward selection

In [23]:
def backward_selection(data,target):
    variate=data.columns.to_list()
    variate.remove(target)  
    selected=variate.copy()
    current_score,best_new_score=float('inf'),float('inf')  
    while variate:
        bic_with_variate=[]
        for feature in variate: 
            l = selected.copy()
            l.remove(feature)
            formula="{}~{}".format(target,"+".join(l))
            print(formula)
            bic=ols(formula=formula,data=data).fit().bic  
            bic_with_variate.append((bic,feature))  
        bic_with_variate.sort(reverse=True)  
        best_new_score,best_feature=bic_with_variate.pop()  
        if current_score >= best_new_score:  
            variate.remove(best_feature)  
            selected.remove(best_feature)  
            current_score=best_new_score  
        else:
            print("selection is over")
            break
    formula="{}~{}".format(target,"+".join(selected))  
    print("final formula is {}".format(formula))
    model=ols(formula=formula,data=data).fit()
    return(model)


In [24]:
bwd = backward_selection(data=data2,target="lpsa")
bwd.summary()

lpsa~lweight+age+lbph+svi+lcp+gleason+pgg45
lpsa~lcavol+age+lbph+svi+lcp+gleason+pgg45
lpsa~lcavol+lweight+lbph+svi+lcp+gleason+pgg45
lpsa~lcavol+lweight+age+svi+lcp+gleason+pgg45
lpsa~lcavol+lweight+age+lbph+lcp+gleason+pgg45
lpsa~lcavol+lweight+age+lbph+svi+gleason+pgg45
lpsa~lcavol+lweight+age+lbph+svi+lcp+pgg45
lpsa~lcavol+lweight+age+lbph+svi+lcp+gleason
lpsa~lweight+age+lbph+svi+lcp+pgg45
lpsa~lcavol+age+lbph+svi+lcp+pgg45
lpsa~lcavol+lweight+lbph+svi+lcp+pgg45
lpsa~lcavol+lweight+age+svi+lcp+pgg45
lpsa~lcavol+lweight+age+lbph+lcp+pgg45
lpsa~lcavol+lweight+age+lbph+svi+pgg45
lpsa~lcavol+lweight+age+lbph+svi+lcp
lpsa~lweight+age+lbph+svi+pgg45
lpsa~lcavol+age+lbph+svi+pgg45
lpsa~lcavol+lweight+lbph+svi+pgg45
lpsa~lcavol+lweight+age+svi+pgg45
lpsa~lcavol+lweight+age+lbph+pgg45
lpsa~lcavol+lweight+age+lbph+svi
lpsa~lweight+age+lbph+svi
lpsa~lcavol+age+lbph+svi
lpsa~lcavol+lweight+lbph+svi
lpsa~lcavol+lweight+age+svi
lpsa~lcavol+lweight+age+lbph
lpsa~lweight+lbph+svi
lpsa~lcavol+lbph

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   lpsa   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.624
Method:                 Least Squares   F-statistic:                     54.15
Date:                Tue, 11 May 2021   Prob (F-statistic):           2.44e-20
Time:                        13:31:03   Log-Likelihood:                -102.05
No. Observations:                  97   AIC:                             212.1
Df Residuals:                      93   BIC:                             222.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.7772      0.623     -1.247      0.215      -2.014       0.460
lcavol         0.5259      0.075      7.024      0.000       0.377       0.675
lweight        0.6618      0.176      3.768      0.000       0.313       1.011
svi            0.6657      0.207      3.214      0.002       0.254       1.077
==============================================================================
Omnibus:                        0.099   Durbin-Watson:                   1.501
Prob(Omnibus):                  0.952   Jarque-Bera (JB):                0.215
Skew:                          -0.069   Prob(JB):                        0.898
Kurtosis:                       2.815   Cond. No.                         36.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Ridge regression

In [40]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.model_selection import cross_val_score
ridge_cv = RidgeCV(normalize=True, alphas=np.logspace(-10, 1, 400))
ridge_model = ridge_cv.fit(X_train, y_train)
ridge_prediction = ridge_model.predict(X_test)
ridge_mae = np.mean(np.abs(y_test - ridge_prediction))
ridge_coefs = dict(
    zip(['Intercept'] + data2.columns.tolist()[1:-2],
        np.round(np.concatenate((ridge_model.intercept_, ridge_model.coef_),
                                axis=None), 3))
)

print('Ridge Regression MAE: {}'.format(np.round(ridge_mae, 3)))
print('Ridge Regression coefficients:', ridge_coefs)

Ridge Regression MAE: 0.517
Ridge Regression coefficients: {'Intercept': 0.155, 'lweight': 0.51, 'age': 0.605, 'lbph': -0.016, 'svi': 0.14, 'lcp': 0.692, 'gleason': -0.134}


# Lasso Regression

In [41]:
lasso_cv = LassoCV(normalize=True, alphas=np.logspace(-10, 1, 400))
lasso_model = lasso_cv.fit(X_train, y_train)
lasso_prediction = lasso_model.predict(X_test)
lasso_mae = np.mean(np.abs(y_test - lasso_prediction))
lasso_coefs = dict(
    zip(['Intercept'] + data2.columns.tolist()[1:-2],
        np.round(np.concatenate((lasso_model.intercept_, lasso_model.coef_), axis=None), 3))
)

print('LASSO MAE: {}'.format(np.round(lasso_mae, 3)))
print('LASSO coefficients:', lasso_coefs)

LASSO MAE: 0.523
LASSO coefficients: {'Intercept': 0.429, 'lweight': 0.577, 'age': 0.614, 'lbph': -0.019, 'svi': 0.145, 'lcp': 0.737, 'gleason': -0.206}
